# Understanding LLM APIs with W&B
We will explore OpenAI models API to generate text.

In [1]:
import os
import openai
import tiktoken
import wandb
from pprint import pprint
from getpass import getpass
from wandb.integration.openai import autolog

Set the OpenAI API key

In [2]:
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window.')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api")
  openai.api_key = os.getenv("OPENAI_API_KEY", "")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

OpenAI API key configured


Enable W&B autologging to track our experiments.

In [3]:
# start logging to W&B
autolog({"project":"llmapps", "job_type": "introduction"})

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Network error (ConnectionError), entering retry loop.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


# Tokenization

In [4]:
encoding = tiktoken.encoding_for_model("text-davinci-003")
enc = encoding.encode("Weigths & Biases is awesome!")
print(enc)
print(encoding.decode(enc))

[1135, 328, 9998, 1222, 8436, 1386, 318, 7427, 0]
Weigths & Biases is awesome!


we can decode the tokens one by one

In [5]:
for token_id in enc:
  print(f"{token_id}\t{encoding.decode([token_id])}")

1135	We
328	ig
9998	ths
1222	 &
8436	 Bi
1386	ases
318	 is
7427	 awesome
0	!


# Sampling
Let's sample some text from the model. For this, let's create a wrapper function around the temperature parameters.
Higher temperature will result in more random samples.

In [6]:
def generate_with_temperature(temp):
  "Generate text with a given temperature, higher temperature means more randomness"
  response = openai.Completion.create(
    model="text-davinci-003",
    prompt="Say something about Weights & Biases",
    max_tokens=50,
    temperature=temp,
  )
  return response.choices[0].text

In [7]:
for temp in [0, 0.5, 1, 1.5, 2]:
  pprint(f'TEMP: {temp}, GENERATION: {generate_with_temperature(temp)}')

('TEMP: 0, GENERATION: \n'
 '\n'
 'Weights & Biases is an amazing tool for tracking and analyzing machine '
 'learning experiments. It provides powerful visualizations and insights into '
 'model performance, enabling data scientists to quickly identify areas of '
 'improvement and optimize their models.')
('TEMP: 0.5, GENERATION: \n'
 '\n'
 'Weights & Biases is a powerful tool for tracking, analyzing, and visualizing '
 'machine learning experiments. It provides a comprehensive suite of features '
 'that make it easy to track, compare, and optimize ML models. With its '
 'intuitive UI and powerful')
('TEMP: 1, GENERATION: \n'
 '\n'
 'Weights & Biases is an amazing tool to help with machine learning '
 'experimentation and analysis. It makes it easier to keep track of '
 'experiments, compare models, and track performance metrics over time. It '
 'also has built-in visualizations which make')
('TEMP: 1.5, GENERATION: \n'
 '\n'
 'Weights & Biases is a smart AI platform that helps simpli

You can also use the `top_p` parameter to control the diversity of the generated text. This parameter controls the cumulative probability of the next token. For example, if `top_p=0.9`, the model will pick the next token from the top 90% most likely tokens. The higher the top_p the more likely the model will pick a token that it hasn't seen before. You should only use one of `temperature` or `top_p` at a given time.

In [8]:
def generate_with_topp(topp):
  "Generate text with a given top-p, higher top-p means more randomness"
  response = openai.Completion.create(
    model="text-davinci-003",
    prompt="Say something about Weights & Biases",
    max_tokens=50,
    top_p=topp,
    )
  return response.choices[0].text

In [9]:
for topp in [0.01, 0.1, 0.5, 1]:
  pprint(f'TOP_P: {topp}, GENERATION: {generate_with_topp(topp)}')

('TOP_P: 0.01, GENERATION: \n'
 '\n'
 'Weights & Biases is an amazing tool for tracking and analyzing machine '
 'learning experiments. It provides powerful visualizations and insights into '
 'model performance, enabling data scientists to quickly identify areas of '
 'improvement and optimize their models.')
('TOP_P: 0.1, GENERATION: \n'
 '\n'
 'Weights & Biases is an amazing tool for tracking and analyzing machine '
 'learning experiments. It provides powerful visualizations and insights into '
 'model performance, enabling data scientists to quickly identify areas of '
 'improvement and optimize their models.')
('TOP_P: 0.5, GENERATION: \n'
 '\n'
 'Weights & Biases is an incredibly powerful tool for tracking and visualizing '
 'machine learning experiments. It allows users to quickly and easily compare '
 'different models and track their performance over time, helping them make '
 'better decisions about which models to use and how')
('TOP_P: 1, GENERATION: \n'
 '\n'
 'Weights & B

# Chat API
Let's switch to chat mode and see how the model responds to our messages. We have some control over the model's response by passing a system-role, here we can steer to model to adhere to a certain behaviour.
> We are using gpt-3.5-turbo, this model is faster and cheaper than davinci-003 

In [10]:
MODEL = "gpt-3.5-turbo"
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Say something about Weights & Biases"},
    ],
    temperature=0,
)

response

<OpenAIObject chat.completion id=chatcmpl-8DaY1AMBZjascQo8h91lUwEO4iTXV at 0x1789a7f1990> JSON: {
  "id": "chatcmpl-8DaY1AMBZjascQo8h91lUwEO4iTXV",
  "object": "chat.completion",
  "created": 1698249445,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Weights & Biases is a powerful tool for machine learning experimentation and collaboration. It provides a platform to track and visualize experiments, making it easier to understand and iterate on models. With features like hyperparameter sweeps and experiment comparison, it helps researchers and data scientists optimize their models and make informed decisions. Additionally, Weights & Biases offers integrations with popular machine learning frameworks, making it seamless to incorporate into existing workflows."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 25,
    "completion_tokens": 84,
    "total_tokens": 1

As you can see above, the response is a JSON object with relevant information about the request.

In [11]:
pprint(response.choices[0].message.content)

('Weights & Biases is a powerful tool for machine learning experimentation and '
 'collaboration. It provides a platform to track and visualize experiments, '
 'making it easier to understand and iterate on models. With features like '
 'hyperparameter sweeps and experiment comparison, it helps researchers and '
 'data scientists optimize their models and make informed decisions. '
 'Additionally, Weights & Biases offers integrations with popular machine '
 'learning frameworks, making it seamless to incorporate into existing '
 'workflows.')


In [12]:
wandb.finish()

usage/completion_tokens,▁▂▂▂▂▁▁▂▂█
usage/elapsed_time,▁▁▁▁▁▁▁▁▁▁
usage/prompt_tokens,▁▁▁▁▁▁▁▁▁█
usage/total_tokens,▁▂▂▂▂▁▁▂▂█
usage/completion_tokens,84
usage/elapsed_time,0.0
usage/prompt_tokens,25
usage/total_tokens,109
